In [1]:
import json

# https://public.opendatasoft.com/explore/dataset/europe-road/export/?refine.icc=BE
MAP_FILENAME = "europe-road.geojson"

with open(MAP_FILENAME, "r") as map_file:
    road_map = json.load(map_file)

roads = road_map["features"]
for r in roads:
    r["properties"] = {}
print(len(roads))

460111


In [90]:
import geojsonio

def prepare_geojson(subset):
    # Prepare empty geojson
    geojson = {"type": road_map["type"], "features": []}
    for i in subset:
        road_segment = road_map["features"][i]
        road_segment["properties"] = {}
        geojson["features"].append(road_segment)
    return geojson
    
def display_geojson_subset(subset):
    """Displays the subset of roads (given by the indicies)."""
    geojsonio.display(json.dumps(prepare_geojson(subset)))

def dump_geojson_to_file(subset, filename):
    with open(filename, "w") as f:
        json.dump(prepare_geojson(subset), f)

In [38]:
display_geojson_subset([10000])

In [34]:
def normalize_point(p):
    """
    Since coordinates are written as floating point numbers in the geojson
    we normalize them so that we can easily use the coordinates as indices.
    """
    def normalize_coordinate(x):
        return round(x * 1e6)
    return normalize_coordinate(p[0]), normalize_coordinate(p[1])

import geopy.distance
def road_len(r):
    coordinates = r["geometry"]["coordinates"]
    ans = 0
    for i in range(len(coordinates) - 1):
        ans += geopy.distance.geodesic(coordinates[i][::-1], coordinates[i + 1][::-1]).km
    return ans

# We represent the graph as an adjacency list.
# For each vertex, we store a list of its neighbors with their lengths.

from tqdm import tqdm

def prepare_graph(roads):
    graph = {}
    for i in tqdm(range(len(roads))):
        r = roads[i]
        l = road_len(r)
        a = normalize_point(r["geometry"]["coordinates"][0])
        b = normalize_point(r["geometry"]["coordinates"][-1])
        
        if a not in graph:
            graph[a] = []
        graph[a].append((b, l, i))
        
        if b not in graph:
            graph[b] = []
        graph[b].append((a, l, i))
    return graph
        
graph = prepare_graph(roads)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 460111/460111 [05:32<00:00, 1381.77it/s]


In [35]:
len(graph)

384335

In [44]:
def random_walk(start, steps):
    prev = None
    cur = start
    subset = []
    for i in range(steps):
        for neighbor, edge_len, edge_index in graph[cur]:
            if neighbor != prev:
                prev = cur
                cur = neighbor
                subset.append(edge_index)
                break
    return subset

In [45]:
display_geojson_subset(random_walk(list(graph.keys())[0], 100))

In [124]:
from queue import PriorityQueue

def vertex_distance(a, b):
    return geopy.distance.geodesic((a[1] / 1e6, a[0] / 1e6), (b[1] / 1e6, b[0] / 1e6)).km

from scipy.spatial import ConvexHull

def dijkstra(start, goal):
    visited = set()
    queue = PriorityQueue()
    queue.put((0, 0, start))
    best_dist = {start: 0}
    prev_vertex = {}
    
    while not queue.empty():
        heuristic, distance, vertex = queue.get()
        if vertex in visited:
            continue
        visited.add(vertex)
        
        if vertex == goal:
            subset = []
            while vertex != start:
                subset.append(prev_vertex[vertex][1])
                vertex = prev_vertex[vertex][0]
            
            coordinates = []
            print("Visited: ", len(visited))
            for v in visited:
                coordinates.append((v[0] / 1e6, v[1] / 1e6))
                
            hull = list(ConvexHull(coordinates).vertices)
            hull.append(hull[0])
            
            hull_points = []
            for i in hull:
                hull_points.append(coordinates[i])
    
            geojson = {
                "type": "FeatureCollection",
                "features": [
                    {
                        "type": "Feature",
                        "properties": {},
                        "geometry": {
                            "type": "LineString",
                            "coordinates": hull_points
                        }
                    }
                ]
            }
        
            with open("visited.json", "w") as f:
                json.dump(geojson, f)

            
            return distance, subset
        
        for neighbor, edge_len, edge_index in graph[vertex]:
            if neighbor in visited:
                continue
            new_distance = distance + edge_len
            # A* heuristics
            heuristic = new_distance + vertex_distance(neighbor, goal) 
            
            if neighbor not in best_dist or best_dist[neighbor] > new_distance:
                best_dist[neighbor] = new_distance
                queue.put((heuristic, new_distance, neighbor))
                # We could remove the original record the the queue but it
                # is about the same amount of work if we just leave it there
                # and discard it when it bubbles to the top (because the vertex
                # is already going to be marked as "visited at that point").
                prev_vertex[neighbor] = vertex, edge_index
    
    return None
    

In [70]:
start = list(graph.keys())[0]
goal = list(graph.keys())[98765]

distance, subset = dijkstra(start, goal)
print(distance)
print(len(subset))
display_geojson_subset(subset[:300])

595.4863247323506
247


In [82]:
def find_vertex(coordinate):
    """Finds a vertex closest to the given coordinate."""
    vertices = list(graph.keys())
    c = normalize_point(coordinate)
    def dist(a, b): return abs(a[0] - b[0]) + abs(a[1] - b[1])
    best = vertices[0]
    best_dist = dist(best, c)
    for v in vertices:
        d = dist(v, c)
        if d < best_dist:
            best = v
            best_dist = d
    return best

PRAGUE = find_vertex((14.412032, 50.086127))
ROME = find_vertex((12.495298, 41.901819))
print("Prague:", PRAGUE)
print("Rome:", ROME)

Prague: (14407397, 50097452)
Rome: (12498114, 41910903)


In [125]:
ans = dijkstra(PRAGUE, ROME)
if ans is None:
    print("No path found :-(")
else:
    distance, subset = ans
    print(distance)
    print(len(subset))
    dump_geojson_to_file(subset, "out.json")

Visited:  46657
1212.6006549726649
876


In [78]:
display_geojson_subset(list(range(1, len(roads), 2000)))

In [80]:
subset = []
for i in tqdm(range(len(roads))):
    r = roads[i]
    c = r["geometry"]["coordinates"][0]
    distance_from_rome = geopy.distance.geodesic(c[::-1], (41.901819, 12.495298)).km
    if distance_from_rome < 10:
        subset.append(i)
print(len(subset))
display_geojson_subset(subset)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 460111/460111 [01:32<00:00, 4974.51it/s]


167


In [115]:
from scipy.spatial import ConvexHull
import numpy

In [119]:
list(ConvexHull([(0, 1), (1, 1), (0, 0), (1, 0), (0.5, 0.5), (10, 10)]).vertices)

[5, 0, 2, 3]